In [ ]:
# Mounting Google Drive Account
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

#Installing bs4 OCR tool
!pip install beautifulsoup4
!apt-get install ocrmypdf
!apt install ghostscript python3-tk
# Importing Packages
import pandas as pd
import os
import time
from os import path
from glob import glob
import requests as req
from bs4 import BeautifulSoup
import subprocess


Mounted at /gdrive
/gdrive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript icc-profiles-free libgs9
  libgs9-common libidn12 libijs-0.35 libimagequant0 libjbig2dec0 libqpdf28 libraqm0 mailcap
  mime-support pngquant poppler-data python3-bs4 python3-chardet python3-coloredlogs
  python3-html5lib python3-humanfriendly python3-img2pdf python3-lxml python3-olefile
  python3-packaging python3-pdfminer python3-pikepdf python3-pil python3-pluggy python3-renderpm
  python3-reportlab python3-reportlab-accel python3-soupsieve python3-tqdm python3-webencodings
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd unpaper
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x ocrmypdf-doc
  python-watchdog img2pdf poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanes

In [ ]:
# Load the CSV file that includes all ICAO Airports
icaoFilePath = r'/gdrive/My Drive/JetBlueProjects/8260s/Current ICAOs/2024-12-20 4_17pm.csv'
folderPath = '/gdrive/My Drive/JetBlueProjects/8260s/Airports/'
df = pd.read_csv(icaoFilePath)
icaoList = df.iloc[0:, 0].tolist()

In [ ]:
def blue_grabber_8260 (icaos, folderName):
    for icao in icaos:
      countSucc = 0
      countErr = 0
      print('*****************'+icao+'*****************')
      if not os.path.exists(folderPath + folderName + icao):
        file_path = os.makedirs(folderPath + folderName + icao)
      url = (
                    'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/?event=procedure.Results&tab=ndbr&nasrId='
                    + icao
                    + '#searchResultsTop')
                    #'&filtered=1&ndbrOnly=1&orderBy=procedure&orderDirection=desc#searchResultsTop'

      print(url)
      response = req.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, 'html.parser')
          links = soup.select('table tbody tr td:first-child a')
          for link in links:
                  indexCfm = 'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/'
                  linkText = link['href']
                  linkURL = req.get(indexCfm+linkText[9:])
                  soup2 = BeautifulSoup(linkURL.text, 'html.parser')
                  links2 = soup2.select('table tbody tr td:first-child a')
                  if links2 != []:
                      for link2 in links2:
                          print(links2[0]['href'])
                          #print('https://www.faa.gov'+links2[0]['href'])
                          pdfLink = 'https://www.faa.gov'+link2['href']
                          print(pdfLink)
                          linkClick = req.get('https://www.faa.gov'+link2['href'])
                          linkClickName = link2['href'].split('/')[-1].replace('.pdf','')
                          print(linkClickName)
                          if linkClick.status_code == 200:
                              with open(folderPath + folderName + icao+ '/' + linkClickName + '.pdf',  'wb') as file:
                                  # Save this as CSV File
                                  file.write(linkClick.content)
                              countSucc = countSucc + 1
                              print('File downloaded successfully' + ' Sucess Downloads Count is: ' + str(countSucc))
                          else:
                              countErr = countErr + 1
                              print('Error: File failed to download ' + 'Failed Download Count is: ' + str(countErr))
                  else:
                      print(indexCfm+linkText[9:])
                      countErr = countErr + 1
                      print('Error: File failed to download ' + 'Failed Download Count is: ' + str(countErr))
    print('\n\n\n\n\n\nFinal Counts: \nSucessful: ' + str(countSucc) + '\nFailed: ' + str(countErr) )
#blue_ocr(folderPath + folderName + icao)


In [ ]:
blue_grabber_8260(['MCO'],'sample')

NameError: name 'blue_grabber_8260' is not defined

In [ ]:
def blue_ocr(directory_path):
    directory_path = '/gdrive/My Drive/JetBlueProjects/8260s/Airports' + '/' + directory_path
    pdfs = os.listdir(directory_path)
    pdfs = glob(path.join(directory_path, '*pdf'))
    if not os.path.exists(directory_path + '-Clean'):
      os.mkdir(directory_path + '-Clean')
    pdfsClean = [path.replace(directory_path, directory_path + '-Clean') for path in pdfs]




    for pdf, pdfClean in zip(pdfs, pdfsClean):
      gs = ["gs", "-dPDFA=1", "-dBATCH", "-r300x300", "-dNOPAUSE", "-sDEVICE=pdfwrite", f"-sOutputFile={pdfClean}", pdf]
      subprocess.run(gs, check=True, text=True, capture_output=True)
      print('1')

    for pdfClean in pdfsClean:
        ocr = [
          "ocrmypdf",
          "--tesseract-oem", "1",
          "--tesseract-config", "-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz/",
          pdfClean,
          pdfClean,
          "--force-ocr"
            ]
        subprocess.run(ocr, check=True, text=True, capture_output=True)



In [ ]:
#blue_ocr('sampleMCO')